This is the gridworld from example 3.8 in the book where we solve it through implementing dynamic programming. 

## Setup

In [1]:
import numpy as np
import pandas as pd 
from functools import partial
from itertools import permutations

In [2]:
## Parameters 
nrow = 5
ncol = 5 
k = 1  # number of moves between policy iterations
gamma = 0.9  # discount factor

In [3]:
# Setup gameboard 
grid = np.zeros((nrow, ncol))
# Add teleporters - start, end, score
tele_start_l = [(0,1), (0,3)]
tele_end_l   = [(4,1), (2,3)]
tele_score_l = [10   , 5    ]

In [4]:
# Actions - up, down, left, right
actions = [(-1,0), (1,0), (0,-1), (0,1)]

## Policy evaluation 

In [7]:
def is_on_grid(pos): 
    if min(pos) < 0 or pos[0] >= nrow or pos[1] >= ncol:   
        return False
    return True

In [8]:
def eval_action(pos, action): 
    """ move, score, and deal with teleporters
    pos, action are tuples. 
    pos is where you are now
    action is the coordinate change"""
    reward = 0 
    ### teleporter case
    tele_flag = False
    for i in range(len(tele_start_l)):
        if pos == tele_start_l[i]:    
            tele_flag = True
            new_pos = tele_end_l[i]; 
            reward += tele_score_l[i]
    if tele_flag == False: 
        ### no teleporter, make valid move 
        new_pos = tuple(np.add(pos,action))
        if is_on_grid(new_pos) == False:   reward -=1 
        if min(new_pos) < 0:      new_pos = (np.max([0, new_pos[0]]), 
                                             np.max([0, new_pos[1]]))
        elif new_pos[0] >= nrow:  new_pos = ((nrow-1), new_pos[1])
        elif new_pos[1] >= ncol:  new_pos = (new_pos[0], (ncol-1))
    return (new_pos, reward)

In [9]:
def bellman_update(idx, reward_l, new_pos_l,pi):
    return (pi[idx]*(reward_l[idx] + gamma * v[new_pos_l[idx]]))

In [10]:
def update_value_for_square(pos,v,pi):
    tmp = [eval_action(pos, action) for action in actions]
    new_pos_l = [o[0] for o in tmp]
    reward_l  = [o[1] for o in tmp]
    # update the value function
    v[pos] = sum(map(partial(bellman_update, reward_l=reward_l, 
                              new_pos_l = new_pos_l, pi=pi), range(len(pi))))
    return(v)

In [24]:
def policy_evaluation(policy,k=100): 
    v = np.zeros((nrow,ncol))
    pos_l = [(x,y) for x in range(nrow) for y in range(ncol)]
    for i in range(k): 
        for pos in pos_l: 
            pi = policy[pos[0]][pos[1]]
            v = update_value_for_square(pos,v,pi)
    return v

In [25]:
pi_start = [0.25, 0.25, 0.25, 0.25]  # Start policy 
policy = [[pi_start for j in range(ncol)] for i in range(nrow)]  # one for each square
v = policy_evaluation(policy)

In [26]:
v.round(1)

array([[ 3.3,  8.8,  4.4,  5.3,  1.5],
       [ 1.5,  3. ,  2.3,  1.9,  0.5],
       [ 0.1,  0.7,  0.7,  0.4, -0.4],
       [-1. , -0.4, -0.4, -0.6, -1.2],
       [-1.9, -1.3, -1.2, -1.4, -2. ]])

## Policy improvement

Where we define a new policy based on the value function. 

In [30]:
def policy_improvement(v):
    # Define list to hold policy
    policy = [[[0,0,0,0] for j in range(ncol)] for i in range(nrow)] 
    for pos in pos_l: 
        legal_moves = [is_on_grid(tuple(np.add(pos,action))) for action in actions]
        values = [v[tuple(np.add(pos,act))] 
                  if legal else -99999999 for act, legal in zip(actions, legal_moves)]
        max_val = max(values)
        n_max_values = sum([1 if o == max_val else 0 for o in values])
        pi_tmp = [1/n_max_values if o == max_val else 0 for o in values]
        policy[pos[0]][pos[1]] = pi_tmp
    return(policy)

In [31]:
policy = policy_improvement(v)

## Policy iteration 

Where we cycle back and forth between the two. 

In [34]:
pi_start = [0.25, 0.25, 0.25, 0.25]  
policy = [[pi_start for j in range(ncol)] for i in range(nrow)] 
for i in range(10):
    v = policy_evaluation(policy, k=10)
    policy = policy_improvement(v)

In [37]:
v.round(1)

array([[ 14.6,  17.1,  14.6,  13.4,  11.2],
       [ 12.7,  14.6,  12.7,  11.2,  10. ],
       [ 11.1,  12.7,  11.1,  10. ,   8.4],
       [ 10. ,  11.1,  10. ,   8.4,   6.7],
       [  7.1,  10. ,   7.1,   6.7,   5.2]])

In [38]:
policy

[[[0, 0, 0, 1.0],
  [0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
  [0, 0, 1.0, 0],
  [0, 0, 1.0, 0],
  [0, 0, 1.0, 0]],
 [[0.5, 0, 0, 0.5],
  [1.0, 0, 0, 0],
  [0.5, 0, 0.5, 0],
  [1.0, 0, 0, 0],
  [0.5, 0, 0.5, 0]],
 [[0.5, 0, 0, 0.5],
  [1.0, 0, 0, 0],
  [0.5, 0, 0.5, 0],
  [1.0, 0, 0, 0],
  [0.5, 0, 0.5, 0]],
 [[0.5, 0, 0, 0.5],
  [1.0, 0, 0, 0],
  [0.5, 0, 0.5, 0],
  [0.5, 0, 0.5, 0],
  [0.5, 0, 0.5, 0]],
 [[0.5, 0, 0, 0.5],
  [1.0, 0, 0, 0],
  [0.5, 0, 0.5, 0],
  [1.0, 0, 0, 0],
  [0.5, 0, 0.5, 0]]]